In [1]:
import os, glob, librosa, pickle
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense

from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
with open ('data_2_10_mfcc', 'rb') as fp:
    dp = pickle.load(fp)

In [3]:
with open ('label_2_10_mfcc', 'rb') as fp:
    lb = pickle.load(fp)

In [4]:
len(dp)

43584

In [5]:
len(dp[0])

3390

In [6]:
onehot_encoder = OneHotEncoder(sparse=False)

In [7]:
lb = np.array(lb)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(dp,lb,test_size = 0.5, random_state = 42, shuffle = True, stratify = lb)

In [55]:
lbl = y_train.reshape((len(y_train),1))

In [56]:
onehot_encoded = onehot_encoder.fit_transform(lbl)

C:\Users\Gurpreet\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [57]:
onehot_encoded.shape

(21792, 109)

In [58]:
X_train = np.array(X_train)

In [59]:
X_train.shape

(21792, 3390)

In [60]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [61]:
X_train = scaler.transform(X_train)

In [62]:
dnn = Sequential()

dnn.add(Dense(900, input_dim = 3390, activation='relu'))
dnn.add(Dense(300, activation='relu'))

dnn.add(Dense(109, activation='softmax'))

In [63]:
dnn.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [64]:
dnn.fit(X_train, onehot_encoded, epochs = 2)

Epoch 1/2
21792/21792 [==============================] - 23s 1ms/step - loss: 3.1607 - acc: 0.2892
Epoch 2/2
21792/21792 [==============================] - 23s 1ms/step - loss: 1.1284 - acc: 0.7127


In [65]:
X_test = scaler.transform(X_test)

In [66]:
lbl_test = y_test.reshape((len(y_test),1))

In [67]:
onehot_encoded_test = onehot_encoder.fit_transform(lbl_test)

C:\Users\Gurpreet\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [68]:
pred = dnn.predict(X_test)

In [69]:
pred.shape

(21792, 109)

In [70]:
from sklearn.metrics import accuracy_score

In [71]:
lb_test = []
for i in range(len(pred)):
    lb_test.append(np.argmax(pred[i]))

In [72]:
lb_test = np.array(lb_test)

In [73]:
lb_test.shape

(21792,)

In [74]:
y_test.shape

(21792,)

In [75]:
accuracy_score(y_test, lb_test)

0.6873623348017621

In [76]:
from sklearn.metrics import matthews_corrcoef

In [77]:
matthews_corrcoef(y_test, lb_test)

0.6846235708444575

In [78]:
from keras.models import load_model

In [79]:
dnn.save('mfcc_10_dnn.h5')

In [46]:
clf = load_model('mfcc_dnn.h5')

In [47]:
prd = clf.predict(X_test)

In [48]:
tr = []
for i in range(len(prd)):
    tr.append(np.argmax(prd[i]))

In [49]:
tr = np.array(tr)

In [50]:
tr.shape

(21792,)

In [51]:
accuracy_score(y_test, tr)

0.9097834067547724

In [52]:
matthews_corrcoef(y_test, tr)

0.9089601509115438

In [80]:
from sklearn.externals import joblib

In [81]:
joblib.dump(scaler, 'scaler_10_mfcc.save')

['scaler_10_mfcc.save']

In [55]:
sclr = joblib.load('scaler.save')

In [57]:
x_ttt = sclr.transform(X_test)

In [58]:
prr = clf.predict(x_ttt)

In [59]:
tr_2 = []
for i in range(len(prr)):
    tr_2.append(np.argmax(prr[i]))

In [60]:
tr_2 = np.array(tr_2)

In [61]:
accuracy_score(y_test, tr_2)

0.9523678414096917

In [62]:
matthews_corrcoef(y_test, tr_2)

0.9519293236562485

In [63]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [64]:
from sklearn.metrics import confusion_matrix

In [65]:
cf = confusion_matrix(y_test, tr_2)

In [70]:
plt.figure(figsize = (218,218))
sn.heatmap(cf, annot=True)

In [2]:
with open ('data_2_10_mfcc', 'rb') as fp:
    dp = pickle.load(fp)

with open ('label_2_10_mfcc', 'rb') as fp:
    lb = pickle.load(fp)

X_train, X_test, y_train, y_test = train_test_split(dp,lb,test_size = 0.5, shuffle = True, random_state = 42, stratify = lb)

In [3]:
from sklearn.externals import joblib
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_recall_fscore_support


sclr = joblib.load('scaler_10_mfcc.save')
clf = load_model('mfcc_10_dnn.h5')
X_test = sclr.transform(X_test)

pred = clf.predict(X_test)


lb_test = []
for i in range(len(pred)):
    lb_test.append(np.argmax(pred[i]))

lb_test = np.array(lb_test)

acc = accuracy_score(y_test, lb_test)
print("Accuracy : ",acc)

mcc = matthews_corrcoef(y_test, lb_test)
print("MCC : ",mcc)

prf = precision_recall_fscore_support(y_test, lb_test, average='macro')
print("Precission : ", prf[0]," Recall : ", prf[1]," F1 Score : ",prf[2])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Accuracy :  0.6873623348017621
MCC :  0.6846235708444575
Precission :  0.6967970371748611  Recall :  0.6806315121817567  F1 Score :  0.6780223248224306
